<a href="https://colab.research.google.com/github/nawnie/EveryDream2trainer/blob/UI/Train_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EveryDream2 Colab Edition

Check out documentation here: https://github.com/victorchall/EveryDream2trainer#docs

And join the discord: https://discord.gg/uheqxU6sXN

In [ ]:
#@title # Install python 3.10 
#@markdown # This will show a runtime error, it's ok, it's on purpose to restart the kernel to update python.
import os
import time
import sys
from IPython.display import clear_output
import subprocess
from tqdm.auto import tqdm

packages = [
    ('torch==1.13.1+cu117 torchvision==0.14.1+cu117', 'https://download.pytorch.org/whl/cu117'),
    'diffusers[torch]==0.14.0'
]

for package in tqdm(packages, desc='Installing packages', unit='package'):
    if isinstance(package, tuple):
        package_name, extra_index_url = package
        cmd = f"pip install -q {package_name} --extra-index-url {extra_index_url}"
    else:
        cmd = f"pip install -q {package}"
    subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

clear_output()

#@markdown Optional connect Gdrive  But strongly recommended
#@markdown This will let you put all your training data and checkpoints directly on your drive.  Much faster/easier to continue later, less setup time.

#@markdown Creates /content/drive/MyDrive/everydreamlogs/ckpt
Mount_to_Gdrive = True #@param{type:"boolean"} 

if Mount_to_Gdrive:
  from google.colab import drive
  drive.mount('/content/drive')

  !mkdir -p /content/drive/MyDrive/everydreamlogs/ckpt

# Define a custom function to display a progress bar
def display_progress_bar(progress, total, prefix=""):
    sys.stdout.write(f"\r{prefix}[{'=' * progress}>{' ' * (total - progress - 1)}] {progress + 1}/{total}")
    sys.stdout.flush()

total_steps = 10
current_step = 0

!git clone https://github.com/victorchall/EveryDream2trainer.git
current_step += 1
display_progress_bar(current_step, total_steps, "install progress:")



!pip install transformers==4.25.1 --progress-bar on --quiet
current_step += 1
display_progress_bar(current_step, total_steps, "install progress:")


!pip install watchdog --progress-bar on --quiet
current_step += 1
display_progress_bar(current_step, total_steps, "install progress:")

!pip install matplotlib --progress-bar on --quiet
current_step += 1
display_progress_bar(current_step, total_steps, "install progress:")

# Install the alive-package library
!pip install alive-progress --progress-bar on --quiet
current_step += 1
display_progress_bar(current_step, total_steps, "install progress:")


# Install the tqdm library
!pip install tqdm --progress-bar on --quiet
current_step += 1
display_progress_bar(current_step, total_steps, "install progress:")

# Download the py310.sh script
!wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh -q
current_step += 1
display_progress_bar(current_step, total_steps, "install progress:")

# Run the py310.sh script
try:
    output = os.popen('bash ./py310.sh -b -f -p /usr/local 2>&1').read()
    total_lines = len(output.splitlines())
    for i, line in enumerate(output.splitlines()):
        clear_output(wait=True)
        display_progress_bar(i, total_lines, "install progress:")
except Exception as e:
    print(str(e))

current_step += 1
display_progress_bar(current_step, total_steps, "install progress:")

# Install the py310 kernel
!python -m ipykernel install --name "py310" --user > /dev/null 2>&1
current_step += 1
display_progress_bar(current_step, total_steps, "install progress:")

# Clear output
!rm /content/py310.sh
current_step += 1
display_progress_bar(current_step, total_steps, "install progress:")
clear_output()
time.sleep(1) #needed to clear is before kill
os.kill(os.getpid(), 9)
print("\nInstallation completed.")


In [1]:
#@markdown # Finish Install Dependencies into the new python
#@markdown This will take a couple minutes, be patient and watch the output for "DONE!"
from IPython.display import clear_output
import subprocess
from tqdm.auto import tqdm

packages = [
    ('torch==1.13.1+cu117 torchvision==0.14.1+cu117', 'https://download.pytorch.org/whl/cu117'),
    'transformers==4.25.1',
    'diffusers[torch]==0.14.0',
    'pynvml==11.4.1',
    'bitsandbytes==0.35.0',
    'ftfy==6.1.1',
    'aiohttp==3.8.3',
    'compel~=1.1.3',
    'tensorboard>=2.11.0',
    'protobuf==3.20.1',
    'wandb==0.13.6',
    'pyre-extensions==0.0.23',
    'xformers==0.0.16',
    'pytorch-lightning==1.6.5',
    'OmegaConf==2.2.3',
    'numpy==1.23.5',
    'colorama',
    'keyboard',
    'triton',
    'lion-pytorch'
]

for package in tqdm(packages, desc='Installing packages', unit='package'):
    if isinstance(package, tuple):
        package_name, extra_index_url = package
        cmd = f"pip install -q {package_name} --extra-index-url {extra_index_url}"
    else:
        cmd = f"pip install -q {package}"
        
    subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

clear_output()


%cd /content/EveryDream2trainer
!python utils/get_yamls.py
clear_output()
print("DONE! installing dependcies make sure we are using python 3.10.x")
!python --version

DONE! installing dependcies make sure we are using python 3.10.x
Python 3.10.6


In [21]:
#@title Get A Base Model
#@markdown Choose SD1.5, Waifu Diffusion 1.3, SD2.1, or 2.1(512) from the dropdown, or paste your own URL in the box
#@markdown * alternately you can link to a HF repo using NAME/MODEL
#@markdown * link to a set of diffusers on your Gdrive
#@markdown * paste a url, atm there is no support for .safetensors

from IPython.display import clear_output
!mkdir input
%cd /content/EveryDream2trainer
MODEL_LOCATION = "hakurei/waifu-diffusion-v1-3" #@param ["sd_v1-5+vae.ckpt", "hakurei/waifu-diffusion-v1-3", "stabilityai/stable-diffusion-2-1-base", "stabilityai/stable-diffusion-2-1"] {allow-input: true}
if MODEL_LOCATION == "sd_v1-5+vae.ckpt":
  MODEL_LOCATION = "panopstor/EveryDream"

import os

download_path = ""

if ".co" in MODEL_LOCATION or "https" in MODEL_LOCATION or "www" in MODEL_LOCATION: #maybe just add a radio button to download this should work for now
    print("Downloading ")
    !wget $MODEL_LOCATION
    clear_output()
    print("DONE!")
    download_path = os.path.join(os.getcwd(), os.path.basename(MODEL_URL))

else:
  save_name = MODEL_LOCATION

%cd /content/EveryDream2trainer
#@markdown * if you chose to link to a .ckpt Select the correct model version in the drop down menu for conversion

inference_yaml = " "

# Check if the downloaded or copied model is a .ckpt file
#@markdown is the model 1.5 or 2.1 based
if download_path.endswith(".ckpt"):
    model_type = "SD1x" #@param ["SD1x", "SD2_512_base", "SD21"]
    save_path = download_path
    if ".ckpt" in save_name:
        save_name = save_name.replace(".ckpt", "")

    img_size = 512
    upscale_attention = False
    prediction_type = "epsilon"
    if model_type == "SD1x":
        inference_yaml = "v1-inference.yaml"
    elif model_type == "SD2_512_base":
        upscale_attention = True
        inference_yaml = "v2-inference.yaml"
    elif model_type == "SD21":
        upscale_attention = True
        prediction_type = "v_prediction"
        inference_yaml = "v2-inference-v.yaml"
        img_size = 768

    !python utils/convert_original_stable_diffusion_to_diffusers.py --scheduler_type ddim  \
    --original_config_file $inference_yaml \
    --image_size $img_size \
    --checkpoint_path $save_path \
    --prediction_type $prediction_type \
    --upcast_attn False \
    --dump_path $save_name

    # Set the save path to the GDrive directory if cache_to_gdrive is True
    if cache_to_gdrive:
        save_name = os.path.join("/content/drive/MyDrive/everydreamlogs/ckpt", save_name)
if inference_yaml != " ":
  print("Model saved to: " + save_name + ". The " + inference_yaml + " was used!")
print("Model " + save_name + " will be used!")


mkdir: cannot create directory ‘input’: File exists
/content/EveryDream2trainer
/content/EveryDream2trainer
Model hakurei/waifu-diffusion-v1-3 will be used!


# Training

For a more indepth Explanation of each of these paramaters check out /content/EveryDream2trainer/doc.

After youve tried a few models you will find /content/EveryDream2trainer/doc/ATWEAKING.md to be extremly helpful.

In [22]:
from google.colab import runtime
from IPython.display import clear_output
import time
from tqdm import tqdm
import re
import sys
import time
import shutil

#@title  #Run Everydream 2
%cd /content/EveryDream2trainer
#@markdown If you want to use a .json config or upload your own, skip this cell and run the cell below instead
!rm -r /content/EveryDream2trainer/Training_Data
#@markdown * Save logs and output ckpts to Gdrive (strongly suggested)
Save_to_Gdrive = True #@param{type:"boolean"}
#@markdown * Save extra data to help continue training with
Save_optimizer_state = True #@param{type:"boolean"}
#@markdown * Disconnect after training to save Credits 
Disconnect_after_training =  False #@param{type:"boolean"}
#@markdown * Use resume to contnue training you just ran, will also find latest diffusers log in your Gdrive to continue.
resume = False #@param{type:"boolean"}
#@markdown * Name your project so you can find it in your logs
Project_Name = "Db_S-1.5e-6-cd80znr80pf0" #@param{type: 'string'}

#@markdown * The learning rate affects how much "training" is done on the model per training step. It is a very careful balance to select a value that will learn your data and not wreck the model. 
#@markdown Leave this default unless you are very comfortable with training and know what you are doing.

Learning_Rate = 1.5e-6 #@param{type: 'number'}

#@markdown * A learning rate scheduler can change your learning rate as training progresses.

#@markdown I recommend sticking with constant until you are comfortable with general training. 

Schedule = "constant" #@param ["constant", "polynomial", "linear", "cosine"] {allow-input: true}

#@markdown * Resolution to train at (recommend 512).  Higher resolution will require lower batch size (below).
Resolution = 512 #@param {type:"slider", min:256, max:768, step:64}

#@markdown * Batch size is also another "hyperparameter" of itself and there are tradeoffs. It may not always be best to use the highest batch size possible. Once of the primary reasons to change it is if you get "CUDA out of memory" errors where lowering the value may help.

#@markdown * Batch size impacts VRAM use.  8 should work on SD1.x models and 5 for SD2.x models at 512 resolution.  Lower this if you get CUDA out of memory errors. You can check resources on your instance and watch the GPU RAM.

Batch_Size = 5 #@param{type: 'number'}

#@markdown * Gradient accumulation is sort of like a virtual batch size increase use this to increase batch size with out increasing vram usage
#@markdown Increasing from 1 to 2 will have a minor impact on vram use, but more beyond that will not.
#@markdown In colab free teir you can expect the fastest proformance from a batch of 8 and a gradient step of 1
#@markdown This is mostly for use if you are training higher resolution on free tier and cannot increase batch size.

Gradient_steps = 1 #@param{type:"slider", min:1, max:10, step:1}

#@markdown * Location on your Gdrive where your training images are.
Dataset_Location = "/content/drive/MyDrive/2run/DB_Super.zip" #@param {type:"string"}

model = save_name

#@markdown * Max Epochs to train for, this defines how many total times all your training data is used. Default of 100 is a good start if you are training ~30-40 images of one subject. If you have 100 images, you can reduce this to 40-50 and so forth.

Max_Epochs = 200 #@param {type:"slider", min:0, max:200, step:5}

#@markdown * How often to save checkpoints.
Save_every_N_epoch = 50 #@param{type:"integer"}

#@markdown * Test sample generation steps, how often to generate samples during training.

#@markdown You can set your own sample prompts by adding them, one line at a time, to `/content/EveryDream2trainer/sample_prompts.txt`.  If left empty, it will use the captions from your training images.

#@markdown Use the steps_between_samples to set how often the samples are generated.
Steps_between_samples = 190 #@param{type:"integer"}

#@markdown * That's it!  Run the cell! or configure these advance options

#@markdown # ________________ ADV SETTINGS _________________
#@markdown These are the default Every Dream 2 settings, changing these without learning what they do will likley waste compute credits
#@markdown please read the doc folder before changing these!

#@markdown * A tip using the sliders, to finely adjust these click them with your mouse then use your keyboard arrows

#@markdown * Using the same seed each time you train allows for more accurate a/b comparison of models, leave at -1 for random
#@markdown * The seed also effects your training samples, if you want the same seed each sample you will need to change it from -1
Training_Seed = -1 #@param{type:"integer"}
#@markdown * warm up steps are useful for validation and cosine lrs
Lr_warmup_steps = 50 #@param{type:"integer"}
#@markdown * use this option to configure a sample_prompts.json
#@markdown * check out /content/EveryDream2trainer/doc/logging.md. for more details
Advance_Samples = False #@param{type:"boolean"}
Sample_File = "sample_prompts.txt"
if Advance_Samples:
  Sample_File = "sample_prompts.json"
#@markdown * Checkpointing Saves Vram to allow larger batch sizes minor slow down on a single batch size but will can allow room for a higher traning resolution (suggested on Colab Free tier, turn off for A100)
Gradient_checkpointing = True #@param{type:"boolean"}
Disable_Xformers = False #@param{type:"boolean"}
#@markdown * Tag shuffling, mainly for booru training. Best to just read this if interested in shufflng tags /content/EveryDream2trainer/doc/SHUFFLING_TAGS.md
shuffle_tags = False #@param{type:"boolean"}
#@markdown * You can turn off the text encoder training (generally not suggested)
Disable_text_Encoder= False #@param{type:"boolean"}
#@markdown * Skip the nth last layer of CLIP.
Clip_skip = 1 #@param {type:"slider", min:0, max:4, step:1}
#@markdown * ratio of captions dropped from training data.
Conditional_DropOut = 0.08 #@param {type:"slider", min:0, max:0.3, step:0.01}
#@markdown * Ratio of images randomly to flip horizontally.
Picture_flip = 0 #@param {type:"slider", min:0, max:0.5, step:0.05}
#@markdown * This can improve contrast in light and dark scenes, Use a ratio between 0-10% for Best results.
zero_frequency_noise = 0.08 #@param {type:"slider", min:0, max:0.25, step:0.01}

#@markdown * Weights and Biases logging token. 
# #@markdown Paste your token here if you have an account so you can use it to track your training progress.  If you don't have an account, you can create one for free at https://wandb.ai/site.  Log will use your project name from above. This is a free online logging utility.
# #@markdown Your key is on this page: https://wandb.ai/settings under "Danger Zone" "API Keys"
wandb_token = '' #@param{type:"string"}


#@markdown use validation with wandb

validatation = False #@param{type:"boolean"}


extensions = ['.zip', '.7z', '.rar', '.tgz']
uncompressed_dir = 'Training_Data'

if any(ext in Dataset_Location for ext in extensions):
    # Create the uncompressed directory if it doesn't exist
    if not os.path.exists(uncompressed_dir):
        os.makedirs(uncompressed_dir)
    
    # Extract the compressed file to the uncompressed directory
    shutil.unpack_archive(Dataset_Location, uncompressed_dir)

    # Set the dataset location to the uncompressed directory
    Dataset_Location = uncompressed_dir

# Use the dataset location in the rest of your code
dataset = Dataset_Location

Drive=""
if Save_to_Gdrive:
  Drive = "--logdir /content/drive/MyDrive/everydreamlogs --save_ckpt_dir /content/drive/MyDrive/everydreamlogs/ckpt"

if Max_Epochs==0:
  Max_Epoch=1

if resume:
  model = "findlast"

validate = ""
if validatation:
  validate = "--validation_config validation_default.json"

wandb_settings = ""
if wandb_token:
  !wandb login $wandb_token
  wandb_settings = "--wandb"

Gradient = ""
if Gradient_checkpointing:
  Gradient = "--gradient_checkpointing "

DX = ""  
if Disable_Xformers:
  DX = "--disable_xformers "

shuffle = ""
if shuffle_tags:
  shuffle = "--shuffle_tags "

Save_optimizer = ""
if Save_optimizer_state:
  Save_optimizer = "--save_optimizer"

textencode = ""
if Disable_text_Encoder:
  textencode = "--disable_textenc_training"

def parse_progress(log_line):
    match = re.search(r'\((\d+)%\)', log_line)
    if match:
        return int(match.group(1))
    return None
  

def update_progress_bar(progress: float):
    print("Training progress: {:.2f}%".format(progress))
    print("[{0}{1}]".format('#' * int(progress // 2), ' ' * (50 - int(progress // 2))))

# Start the training process and capture the output
command = f"""python3.10 train.py --resume_ckpt "{model}" \
  {textencode} \
  {Gradient} \
  {shuffle} \
  {Drive} \
  {DX} \
  {validate} \
  {wandb_settings} \
  {Save_optimizer} \
  --clip_skip {Clip_skip} \
  --batch_size {Batch_Size} \
  --grad_accum {Gradient_steps} \
  --cond_dropout {Conditional_DropOut} \
  --data_root "{dataset}" \
  --flip_p {Picture_flip} \
  --lr {Learning_Rate} \
  --log_step 25 \
  --lr_warmup_steps {Lr_warmup_steps} \
  --lr_scheduler "{Schedule}" \
  --max_epochs {Max_Epochs} \
  --project_name "{Project_Name}" \
  --resolution {Resolution} \
  --sample_prompts "{Sample_File}" \
  --sample_steps {Steps_between_samples} \
  --save_every_n_epoch {Save_every_N_epoch} \
  --seed {Training_Seed} \
  --zero_frequency_noise_ratio {zero_frequency_noise}"""

process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# Initialize the progress bar
progress_bar = tqdm(total=100, desc="Training progress", ncols=100)

for log_line in process.stdout:
    global last_output
    last_output = None
    log_line = log_line.strip()
    if log_line:
        if log_line != last_output:
            progress = parse_progress(log_line)
            if progress is not None:
                update_progress_bar(progress)
            else:
                print(log_line)
            last_output = log_line

# Finish the training process
process.wait()
if Disconnect_after_training:
    time.sleep(40)
    runtime.unassign()


/content/EveryDream2trainer



Training progress:   0%|                                                    | 0/100 [22:13<?, ?it/s]


** Your branch 'main' is up to date with the remote
No config file specified, using command line args
logging to /content/drive/MyDrive/everydreamlogs/Db_S-1.5e-6-cd80znr80pf0-20230430-104131.log
Args:
{'amp': True,
'aspects': [[512, 512],
[576, 448],
[448, 576],
[640, 384],
[384, 640],
[768, 320],
[320, 768],
[896, 256],
[256, 896],
[1024, 256],
[256, 1024]],
'batch_size': 5,
'ckpt_every_n_minutes': 1000000000.0,
'clip_grad_norm': None,
'clip_skip': 1,
'cond_dropout': 0.08,
'config': None,
'data_root': 'Training_Data',
'disable_amp': False,
'disable_textenc_training': False,
'disable_unet_training': False,
'disable_xformers': False,
'flip_p': 0.0,
'gpuid': 0,
'grad_accum': 1,
'gradient_checkpointing': True,
'log_step': 25,
'logdir': '/content/drive/MyDrive/everydreamlogs',
'lowvram': False,
'lr': 1.5e-06,
'lr_decay_steps': 0,
'lr_scheduler': 'constant',
'lr_warmup_steps': 50,
'max_epochs': 200,
'optimizer_config': 'optimizer.json',
'project_name': 'Db_S-1.5e-6-cd80znr80pf0',
'rated_da

In [6]:
#@title json Editor for ED2

import json

data = {
  "disable_textenc_training": False,
  "disable_xformers": False,
  "disable_amp": False,
  "lowvram": False,
  "notebook": False,
  "save_optimizer": False,
  "scale_lr": False,
  "gradient_checkpointing": True,
  "wandb": False,
  "write_schedule": False,
  "rated_dataset": False,
  "batch_size": 10,
  "ckpt_every_n_minutes": None,
  "clip_grad_norm": None,
  "clip_skip": 0,
  "cond_dropout": 0.04,
  "data_root": "X:\\my_project_data\\project_abc",
  "flip_p": 0.0,
  "gpuid": 0,
  "grad_accum": 1,
  "logdir": "logs",
  "log_step": 25,
  "lr": 1.5e-06,
  "lr_decay_steps": 0,
  "lr_scheduler": "constant",
  "lr_warmup_steps": None,
  "max_epochs": 30,
  "optimizer_config": "optimizer.json",
  "project_name": "project_abc",
  "resolution": 512,
  "resume_ckpt": "sd_v1-5_vae",
  "run_name": None,
  "sample_prompts": "sample_prompts.txt",
  "sample_steps": 300,
  "save_ckpt_dir": None,
  "save_ckpts_from_n_epochs": 0,
  "save_every_n_epochs": 20,
  "seed": 555,
  "shuffle_tags": False,
  "validation_config": "validation_default.json",
  "rated_dataset_target_dropout_percent": 50,
  "zero_frequency_noise_ratio": 0.02
}

%cd /content/EveryDream2trainer
#@markdown JSON Parameters
findlast = "" 
Resume_Last_Training_session = False #@param {type:"boolean"}
findlast == Resume_Last_Training_session
disable_textenc_training = False #@param {type:"boolean"}
data["disable_textenc_training"] = disable_textenc_training
disable_xformers = False #@param {type:"boolean"}
data["disable_xformers"] = disable_xformers
gradient_checkpointing = True #@param {type:"boolean"}
data["gradient_checkpointing"] = gradient_checkpointing
save_optimizer = False #@param {type:"boolean"}
data["save_optimizer"] = save_optimizer 
scale_lr = False #@param {type:"boolean"}
data["scale_lr"] = scale_lr
shuffle_tags = False #@param {type:"boolean"}
data["shuffle_tags"] = shuffle_tags
wandb = False #@param {type:"boolean"}
data["wandb"] = wandb
write_schedule = False #@param {type:"boolean"}
data["write_schedule"] = write_schedule
rated_dataset = False #@param {type:"boolean"}
data["rated_dataset"] = rated_dataset 
batch_size = 8 #@param {type:"integer"}
data["batch_size"] = batch_size
ckpt_every_n_minutes = None #@param {type:"raw"}
data["ckpt_every_n_minutes"] = ckpt_every_n_minutes
clip_grad_norm = None #@param {type:"raw"}
data["clip_grad_norm"] = clip_grad_norm
clip_skip = 0 #@param {type:"integer"}
data["clip_skip"] = clip_skip
cond_dropout = 0.04 #@param {type:"number"}
data["cond_dropout"] = cond_dropout
data_root = "X:\\my_project_data\\project_abc" #@param {type:"string"}
data["data_root"] = data_root
flip_p = 0.0 #@param {type:"number"}
data["flip_p"] = flip_p
grad_accum = 1 #@param {type:"integer"}
data["grad_accum"] = grad_accum
logdir = "logs" #@param {type:"string"}
data["logdir"] = logdir
log_step = 25 #@param {type:"integer"}
data["log_step"] = log_step
lr = 1.5e-06 #@param {type:"number"}
data["lr"] = lr
lr_decay_steps = 0 #@param {type:"integer"}
data["lr_decay_steps"] = lr_decay_steps
lr_scheduler = "constant" #@param {type:"string"}
data["lr_scheduler"] = lr_scheduler
lr_warmup_steps = None #@param {type:"raw"}
data["lr_warmup_steps"] = lr_warmup_steps
max_epochs = 100 #@param {type:"integer"}
data["max_epochs"] = max_epochs
optimizer_config = "optimizer.json" #@param {type:"string"}
data["optimizer_config"] = optimizer_config
project_name = "project_abc" #@param {type:"string"}
data["project_name"] = project_name
resolution = 512 #@param {type:"integer"}
data["resolution"] = resolution
resume_ckpt = "sd_v1-5_vae" #@param {type:"string"}
if findlast:
  resume_ckpt = "findlast"
data["resume_ckpt"] = resume_ckpt
run_name = None #@param {type:"raw"}
data["run_name"] = run_name
sample_prompts = "sample_prompts.txt" #@param ["sample_prompts.txt", "sample_prompts.json"]
data["sample_prompts"] = sample_prompts
sample_steps = 300 #@param {type:"integer"}
data["sample_steps"] = sample_steps
save_ckpt_dir = None #@param {type:"raw"}
data["save_ckpt_dir"] = save_ckpt_dir
save_ckpts_from_n_epochs = 0 #@param {type:"integer"}
data["save_ckpts_from_n_epochs"] = save_ckpts_from_n_epochs
save_every_n_epochs = 20 #@param {type:"integer"}
data["save_every_n_epochs"] = save_every_n_epochs
seed = 555 #@param {type:"integer"}
data["seed"] = seed
validation_config = "validation_default.json" #@param {type:"string"}
data["validation_config"] = validation_config
rated_dataset_target_dropout_percent = 50 #@param {type:"integer"}
data["rated_dataset_target_dropout_percent"] = rated_dataset_target_dropout_percent
zero_frequency_noise_ratio = 0.02 #@param {type:"number"}
data["zero_frequency_noise_ratio"] = zero_frequency_noise_ratio



# Display the modified JSON data
print("Modified JSON data:")
print(json.dumps(data, indent=2))


# Save the modified JSON data to a file
filename = "train.json" #@param {type:"string"}

with open(filename, 'w') as file:
    json.dump(data, file, indent=2)

print(f"Modified JSON data saved to '{filename}'.")

/content/EveryDream2trainer
Modified JSON data:
{
  "disable_textenc_training": false,
  "disable_xformers": false,
  "disable_amp": false,
  "lowvram": false,
  "notebook": false,
  "save_optimizer": false,
  "scale_lr": false,
  "gradient_checkpointing": true,
  "wandb": false,
  "write_schedule": false,
  "rated_dataset": false,
  "batch_size": 8,
  "ckpt_every_n_minutes": null,
  "clip_grad_norm": null,
  "clip_skip": 0,
  "cond_dropout": 0.04,
  "data_root": "X:\\my_project_data\\project_abc",
  "flip_p": 0.0,
  "gpuid": 0,
  "grad_accum": 1,
  "logdir": "logs",
  "log_step": 25,
  "lr": 1.5e-06,
  "lr_decay_steps": 0,
  "lr_scheduler": "constant",
  "lr_warmup_steps": null,
  "max_epochs": 100,
  "optimizer_config": "optimizer.json",
  "project_name": "project_abc",
  "resolution": 512,
  "resume_ckpt": "sd_v1-5_vae",
  "run_name": null,
  "sample_prompts": "sample_prompts.txt",
  "sample_steps": 300,
  "save_ckpt_dir": null,
  "save_ckpts_from_n_epochs": 0,
  "save_every_n_epoch

In [ ]:
#@title Alternate startup script
#@markdown * Edit train.json to setup your paramaters

#@markdown * Edit using a chain length of 0 will use train.json

#@markdown * Edit the above cell saving it as chain1.json to make use of chaining

#@markdown * make sure to check each confguration you will need 1 Json per chain length 3 are provided

#@markdown * make sure your .Json contain the line Notebook: true

#@markdown * your locations in the .json can be done in this format /content/drive/MyDrive/   - then the sub folder you wish to use

%cd /content/EveryDream2trainer
Chain_Length=0 #@param{type:"integer"}
l = Chain_Length 
I=0 #repeat counter
if l == None or l == 0:
  train.py --config train.json
while l > 0:
  !python train.py --config chain{I}.json
  l -= 1
  I =+ 1

In [20]:
#@title Test your Diffusers
import torch
from diffusers import StableDiffusionPipeline, DiffusionPipeline, DPMSolverMultistepScheduler
import torch
#from xformers.ops import MemoryEfficientAttentionFlashAttentionOp

repo_id = "stabilityai/stable-diffusion-2-base"
pipe = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, revision="fp16")

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

prompt = "High quality photo of an astronaut riding a horse in space"
image = pipe(prompt, num_inference_steps=25).images[0]
image.save("astronaut.png")

#@markdown Path to the diffusers that was trained

#@markdown You can look in the file drawer on the left /content/drive/MyDrive/everydreamlogs and click the three dots to copy the path

#@markdown ex. /content/drive/MyDrive/everydreamlogs/my_project_20230126-023804/ckpts/interrupted-gs86

sd2 = True #@param {type:"boolean"}
diffusers_path="stabilityai/stable-diffusion-2-1-base" #@param{type:"string"}

PROMPT= "a photo of an astronaut on the moon" #@param{type:"string"}
number_of_inference_steps= 35 #@param{type:"integer"}
sample_name = "test" #@param{type:"string"}
n_propmt = "bad, deformed, ugly, bad anotomy"

#pipe.load_textual_inversion("sd-concepts-library/cat-toy")
#pipe.enable_xformers_memory_efficient_attention(attention_op=MemoryEfficientAttentionFlashAttentionOp)



if sd2:
  pipe = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, revision="fp16")
else:
  pipe = StableDiffusionPipeline.from_pretrained(diffusers_path, torch_dtype=torch.float16)

pipe = pipe.to("cuda")

image = pipe(PROMPT, num_inference_steps=number_of_inference_steps, negative_prompt=n_propmt).images[0]


image.save(sample_name + ".png")

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]